# <center> <u>**Détections** </u></center>



# 1. Chargement des données

## 1.1. Vériﬁcation des données

On vérifie que le transfert entre le disque dur et le disque local s'est bien passé.

In [20]:
# define variables

path_to_hard_drive = '/media/basile/One Touch/Disque dur Axel pour Basile/BD_71'
path_to_jsons = '/data_bees_detection/BD_71_Annotations/JSON'
path_to_images = '../data_bees_detection/BD_71'
path_to_output = '/datafiles/yolo/inputs'

# define the taxon to detect
taxon_detection = 'Anthophila'

In [2]:
! find $path_to_images -type f | wc -l

111983


En lancant la commande : 
```

! find "$path_to_hard_drive" -type f | wc -l
```


Depuis un terminal sur le disque dur on obtient aussi 111983.

## 1.2. Edition d'un csv récapitulatif

csv avec seulement les chemins des images.

In [19]:
! find $path_to_images -type f >> $path_to_output/BD_71.csv

csv issus des fichiers json avec le format suivant :

* image_path
* xmin
* ymin
* xmax
* ymax
* label
* width
* height

Edité avec le script [suivant](../src/yolo/utils_for_datafiles/json_to_csv.py).


In [21]:
%run /src/yolo/utils_for_datafiles/json_to_csv.py  -j $path_to_jsons -i $path_to_images -t $taxon_detection -o $path_to_output

Visiblement certaines images sont manquantes (référencées dans BD_71_missing_old.csv) : les images sont corrompues ou manquantes.

In [28]:
import pandas as pd
# remplacer les images corrompues par des images saines
path_missing = '/datafiles/yolo/inputs/BD_71_missing.csv'
df_missing = pd.read_csv(path_missing, header=None,skiprows=1)

path_saines = '../data_bees_detection/corrompues_bd71'

df_missing['img_name'] = df_missing[0].apply(lambda x: x.split('/')[-1])
df_missing['new_path'] = df_missing['img_name'].apply(lambda x: path_saines + '/' + x)

In [29]:
df_missing.head()

,0,img_name,new_path
0,/workspaces/projet_bees_detection_basile/data_...,Bombus pascuorum96207.jpeg,/workspaces/projet_bees_detection_basile/data_...
1,/workspaces/projet_bees_detection_basile/data_...,_dsc0461.jpg,/workspaces/projet_bees_detection_basile/data_...
2,/workspaces/projet_bees_detection_basile/data_...,DH9_1898-4.jpg,/workspaces/projet_bees_detection_basile/data_...
3,/workspaces/projet_bees_detection_basile/data_...,DH9_1916-2.jpg,/workspaces/projet_bees_detection_basile/data_...
4,/workspaces/projet_bees_detection_basile/data_...,DH9_1899-2.jpg,/workspaces/projet_bees_detection_basile/data_...


In [31]:
import os

for index, row in df_missing.iterrows():
    
    # copy the image from the saines folder to the BD_71 folder
    os.system('cp "' + row['new_path'] + '" "' + row[0] + '"')

cp: cannot stat '/workspaces/projet_bees_detection_basile/data_bees_detection/corrompues_bd71/Bombus pascuorum96207.jpeg': No such file or directory
cp: cannot stat '/workspaces/projet_bees_detection_basile/data_bees_detection/corrompues_bd71/_dsc0461.jpg': No such file or directory


Vériﬁcation des images manquantes :

In [32]:
%run /src/yolo/utils_for_datafiles/json_to_csv.py  -j $path_to_jsons -i $path_to_images -t $taxon_detection -o $path_to_output

### Parenthèse : pourquoi il y a des images manquantes ?

i.e. pourquoi 111983 images dans BD_71 et seulement 24802 dans BD_71_input.csv ?

Car on ne prend que les images qui ont été vérifiées. 

Cf. la démonstration dans le notebook [suivant](../Notebooks/(old%20_%20check_json).ipynb).

## 1.3. Split des données en train / test / validation

In [34]:
path_to_dataset = '/datafiles/yolo/inputs/BD_71_input.csv'

df_dataset = pd.read_csv(path_to_dataset, header=None)

# shuffle the dataset
df_dataset = df_dataset.sample(frac=1).reset_index(drop=True)

# split the dataset into train, test and validation sets
# 80% train, 10% test, 10% validation

train_size = int(df_dataset.shape[0] * 0.8)
test_size = int(df_dataset.shape[0] * 0.1)
val_size = int(df_dataset.shape[0] * 0.1)

df_train = df_dataset.iloc[:train_size]
df_test = df_dataset.iloc[train_size:train_size+test_size]
df_val = df_dataset.iloc[train_size+test_size:]

# save the datasets
path_to_train = '/datafiles/yolo/inputs/BD_71_train.csv'
path_to_test = '/datafiles/yolo/inputs/BD_71_test.csv'
path_to_val = '/datafiles/yolo/inputs/BD_71_val.csv'

df_train.to_csv(path_to_train, header=None, index=False)
df_test.to_csv(path_to_test, header=None, index=False)
df_val.to_csv(path_to_val, header=None, index=False)

# 2. Entraînement du modèle

## 2.1 Générer des anchor box

On pourra utiliser ce [template](../datafiles/yolo/configs/benchmark_configs/example.json) de fichier de conﬁguration pour commencer.

Le script [suivant](../src/yolo/utils/gen_anchors.py) permet de générer des anchor box.

In [6]:
config_path = '/datafiles/yolo/configs/benchmark_configs/example.json'
nb_anchors = 5


In [9]:
%run /src/yolo/utils/gen_anchors.py -c $config_path -a $nb_anchors

  0%|          | 0/19841 [00:00<?, ?it/s]

100%|██████████| 19841/19841 [00:00<00:00, 189654.77it/s]
2023-07-05 21:30:14.222848: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-05 21:30:14.222877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: f65f42a282a8
2023-07-05 21:30:14.222883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: f65f42a282a8
2023-07-05 21:30:14.222982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2023-07-05 21:30:14.223073: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 510.108.3


feature extractor output shape: (7, 7)
grid size: 32.0 x 32.0
iteration 1: dists = 53781.054326253776
iteration 2: dists = 5689.58627195336
iteration 3: dists = 981.375237331076
iteration 4: dists = 489.63697201631794
iteration 5: dists = 380.94913897540494
iteration 6: dists = 330.63857840727735
iteration 7: dists = 247.7071562017945
iteration 8: dists = 227.56105462402465
iteration 9: dists = 186.2180919749961
iteration 10: dists = 168.30232192469404
iteration 11: dists = 116.4734374365683
iteration 12: dists = 101.77870792695333
iteration 13: dists = 84.65858050382136
iteration 14: dists = 83.10064015129528
iteration 15: dists = 73.18377370377317
iteration 16: dists = 43.71904567227098
iteration 17: dists = 30.83243374718775
iteration 18: dists = 30.9869460556548
iteration 19: dists = 27.996159037650276
iteration 20: dists = 21.35436263309002
iteration 21: dists = 9.186860488539484
iteration 22: dists = 5.62319127221101
iteration 23: dists = 11.149817728351678
iteration 24: dists = 

On peut reporter les anchor box dans le fichier de conﬁguration.

## 2.2. Edition du fichier de configuration

Ajuster au besoin les paramètres du fichier de conﬁguration.

## 2.3. Entraînement du modèle

Voir [script](../src/yolo/train.py).

In [ ]:
config_path = '/datafiles/yolo/configs/benchmark_configs/example.json'

In [ ]:
%run ../src/yolo/train.py -c $config_path

# 3. Evaluation du modèle

Avec ce [script](../src/yolo/evaluate.py).

In [15]:
config_path = '../datafiles/yolo/configs/benchmark_configs/example.json'
weights_path = '../datafiles/yolo/saved_weights/Best_model_bestLoss.h5'

In [ ]:
%run ../src/yolo/evaluate.py -c $config_path -w $weights_path

<p align="center">
    <strong> Pour les poids et les configurations ci-dessus, on obtient ces résultats </strong>
  <br>

  <img src="../datafiles/imgs_for_readme/results_yolo.png" alt="résultats" />
</p>

# 4. Pipeline prédictions

Avec ce [script](../src/yolo/predict.py).

Qui prend les arguments suivants :

* -c : chemin vers le fichier de configuration
* -w : chemin vers les poids du modèle
* -i : chemin vers le dossier d'images, la vidéo ou le csv contenant les chemins des images à prédire
* -o : mode de sortie : "image" ou "csv_input"



On lancerait par exemple :

```bash
python predict.py -c ../datafiles/yolo/configs/benchmark_configs/example.json -w ../datafiles/yolo/weights/benchmark_weights/example.h5 -i ../datafiles/yolo/images/ -o csv_input
```

# 5. Crop des prédictions

In [ ]:
import os 


csv_path = "/datafiles/crop/predictions_whole_dataset_cleaned_only_species_50_1000.csv"
cmd = "python3 /src/crop/crop_from_csv.py  -c '" + csv_path + "' -t ../data_bees_detection/cropped_whole_dataset_cleaned_only_species_50_1000  -s  ../data_bees_detection/whole_dataset_cleaned"

os.system(cmd)